In [ ]:
# linear algebra
import numpy as np 
from random import randint


# data processing
import pandas as pd 
from sklearn import preprocessing

# sql query
import pyodbc 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

from sklearn.model_selection import train_test_split


# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score
from sklearn.inspection import permutation_importance


from sklearn import datasets
from sklearn.model_selection import validation_curve

from sklearn.preprocessing import StandardScaler
import imblearn

# roc curve and auc score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve

# The data 

3 seperate data files : The customers, Their journey, The categories of the items

In [ ]:
data = pd.read_csv('FraudAnalysisData_complete.csv')

# Cleaning and excluding 'asd' devices

In [ ]:
data.columns

Index(['OrderId', 'CustomerId', 'DeviceId', 'MobilePhone', 'DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'Lat', 'Lng', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime', 'CountOrders',
       'CountDiscount', 'FirstOrder'],
      dtype='object')

In [ ]:
x =data[data['DeviceId']=='asd']
y = data[data['DeviceId']!='asd']
z = y[y['CustomerId'].isin(x['CustomerId'])]
print(
    x['CustomerId'].nunique(),
    z['CustomerId'].nunique(),
    y['CustomerId'].nunique(),
)
data_cleaned = data.drop(
    data[(data['CustomerId'].isin(z['CustomerId'])) & (data['DeviceId']=='asd')].index
)
DeviceId = data_cleaned.drop(columns = 
                            ['OrderId', 'DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'Lat', 'Lng', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime',
       'CountDiscount', 'FirstOrder']
                            ).drop_duplicates()

###
print(
    DeviceId['CustomerId'].nunique() ,
    DeviceId['MobilePhone'].nunique(),
    DeviceId['DeviceId'].nunique() , 
    DeviceId.shape[0])

###
MultiNumbers = DeviceId.groupby('DeviceId').agg({
    'CustomerId':'nunique'
}).reset_index().drop_duplicates().rename(columns = {'CustomerId':'NumOfCustomerIds'})

###
MultiDevice = DeviceId.groupby('CustomerId').agg({
    'DeviceId':'nunique' , 'CountOrders':'first'
}).reset_index().drop_duplicates().rename(columns = {'DeviceId':'NumOfDevices'})

###
    # MultiNumbers.sort_values(by = 'NumOfCustomerIds' , ascending = False)
NormalDevices = MultiNumbers[MultiNumbers['NumOfCustomerIds']<4]

print(NormalDevices.DeviceId.nunique() , DeviceId.DeviceId.nunique() , 
     NormalDevices.NumOfCustomerIds.sum() , DeviceId.CustomerId.nunique())
# Normaldata_cleaned = data_cleaned[data_cleaned['DeviceId'].isin(NormalDevices['DeviceId'])]

22842 22491 3579037
3579388 3579388 936847 3883403
793366 936847 1076623 3579388


In [ ]:
FraudDevices = MultiNumbers[MultiNumbers['NumOfCustomerIds']>10]

In [ ]:
data_cleaned['FraudId'] = (data_cleaned['DeviceId'].isin(FraudDevices['DeviceId'])).astype(int)

In [ ]:
data_cleaned['Basket'] = data_cleaned['ProductId'].apply(lambda x: x.strip("[]"))

In [ ]:
Devices = data_cleaned.DeviceId.dropna()

In [ ]:
data_cleaned_devices = data_cleaned[data_cleaned.DeviceId.isin(Devices)]

In [ ]:
data_cleaned_devices.iloc[10]['Basket']

'14532,18152,143606,17300,182225,394,171187,18534'

In [ ]:
a = data.drop(columns=['OrderId', 'CustomerId', 'DeviceId', 'MobilePhone', 'DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime', 'CountOrders',
       'CountDiscount', 'FirstOrder'])
print(a.drop_duplicates().shape , a.columns)
a = data.drop(columns=['OrderId', 'DeviceId', 'MobilePhone', 'DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime', 'CountOrders',
       'CountDiscount', 'FirstOrder'])
print(a.drop_duplicates().shape , a.columns)

(4795450, 2) Index(['Lat', 'Lng'], dtype='object')
(4947272, 3) Index(['CustomerId', 'Lat', 'Lng'], dtype='object')


In [ ]:
data.drop(columns='DeviceId').drop_duplicates().shape

(10483083, 23)

In [ ]:
data_cleaned.drop(columns='DeviceId').drop_duplicates().shape

(10483083, 23)

In [ ]:
(data.DeviceId == np.nan).astype(int).sum()

0

In [ ]:
data['CustomerId'].nunique()

3579388

In [ ]:
df = pd.read_csv('df_historical.csv')

In [ ]:
DeviceId = temp.drop(columns = ['DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'Lat', 'Lng', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime', 'CountOrders',
       'CountDiscount', 'FirstOrder'])

In [ ]:
Frauds = temp.drop(columns = [ 'ProductId',
                              'Lat', 'Lng',
       'CreateToShippingDiffDay', 'ShippingTime'])

In [ ]:
Basket_info = pd.read_csv('Location_BasketTime.csv')

In [ ]:
Data['ToMinPrice'] = (Data['OrderAmount'] - Data['CACMinOrderAmount'])/Data['OrderAmount']
Data['DisProfit'] = Data['CACDiscountAmount']*100/Data['OrderAmount']
Data['PaidAmount'] = (Data['OrderAmount'] - Data['CACDiscountAmount'])*100/Data['OrderAmount']
Data['FraudId'] = Data['IsFraud'].astype(int)

In [ ]:
temp[temp['StateCode']==5]

,OrderId,CustomerId,DeviceId,MobilePhone,DiscountProduct,OrderTotal,BuyBacketPrice,YearMonth,CreatedTime,Day,...,CouponDiscount,Lat,Lng,BasketTime,StateCode,CreateToShippingDiffDay,ShippingTime,CountOrders,CountDiscount,FirstOrder
0,1645942,734675,NaN,9140535709,210972.0,1505053.0,1716025.0,202003,22:09:52.4500000,26,...,0.0,30.275043,57.065349,1274.0,5,1,20:00 - 22:00,2,0,NaN
2,1646124,336835,aad251423ff31bf1,9203714141,534170.0,6527530.0,7061700.0,202003,00:03:53.8366667,27,...,600000.0,35.749939,51.335777,19135.0,5,1,14:00 - 16:00,39,15,False
3,1646126,160608,ec43dcfe8b2d9796,9166506750,430350.0,1997150.0,2427500.0,202003,00:03:58.4200000,27,...,0.0,35.814766,50.931739,4998.0,5,1,12:00 - 14:00,10,4,NaN
4,1645908,46404,625553782473c822,9334208198,107750.0,1009550.0,1117300.0,202003,22:01:02.9266667,26,...,160000.0,35.712904,51.498133,4161.0,5,1,18:00 - 20:00,22,17,False
5,1645943,756139,bfc606e3f9e54d1c,9182259100,132050.0,1489450.0,1621500.0,202003,22:09:57.2533333,26,...,250000.0,34.354157,47.071433,1667.0,5,1,10:00 - 12:00,13,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12546135,13786401,4232501,NaN,9155143529,109300.0,535700.0,645000.0,202104,19:55:14.2900000,5,...,0.0,30.293869,57.043889,14.0,5,0,21:00 - 23:00,9,4,NaN
12546139,13786467,1322025,4630e9772538de5c,9163974424,178050.0,1033450.0,1211500.0,202104,19:56:21.0900000,5,...,0.0,33.505067,48.356565,128.0,5,0,21:00 - 23:00,29,6,NaN
12546140,13786467,1322025,43875db5b1af07e0,9163974424,178050.0,1033450.0,1211500.0,202104,19:56:21.0900000,5,...,0.0,33.505067,48.356565,128.0,5,0,21:00 - 23:00,29,6,NaN
12546142,13786535,288809,NaN,9372638527,379000.0,1221000.0,1600000.0,202104,19:57:46.9566667,5,...,0.0,30.355400,48.295566,5.0,5,0,21:00 - 23:00,9,0,NaN


In [ ]:
FirstCoupon = temp.drop(columns = [ 'CustomerId', 'DeviceId', 'MobilePhone', 'DiscountProduct',
       'OrderTotal', 'BuyBacketPrice', 'YearMonth', 'CreatedTime', 'Day',
       'IsFraud', 'MinimumFactoreAmount', 'DiffToNextOrderMinute', 'ProductId',
       'CouponDiscount', 'Lat', 'Lng', 'BasketTime', 'StateCode',
       'CreateToShippingDiffDay', 'ShippingTime', 'CountOrders',
       'CountDiscount'])
FirstCoupon['FirstOrder'] = FirstCoupon['FirstOrder'].fillna(-1)
FirstCoupon['FirstOrder'] = FirstCoupon['FirstOrder'].astype(int)

In [ ]:
temp.loc[0]['ProductId']

'[48665,15426,6286,14549,12539,2865,16338,17263]'

In [ ]:
df

,STD_DI,OrderId,FraudId,STD,DI,NumDis_NumPur,NumDis_DI,NumDis_STD,NumDis_STD_DI,FraudId_avg,DIsum,STD_avg,STD_DI_avg,CustomerId,TimeToShoppingCart,TimeToPreviousOrder,AvgBasketForming,AvgTimeBetweenOrders
0,60.192635,10261118,0,0.957427,2.661840,1.000000,0.899219,29.750568,40.128423,0,2.661840,0.957427,60.192635,5608657,1335,0.0,729.500000,10560.500000
1,56.388985,10861839,0,1.366260,1.648847,0.800000,0.695645,56.291651,46.990821,0,1.648847,1.366260,56.388985,2839661,2,63.0,12.600000,26204.200000
2,55.056967,11216905,0,0.421637,2.274241,1.000000,1.766992,31.158581,55.056967,0,2.274241,0.421637,55.056967,4892643,1470,40138.0,1117.333333,13826.000000
3,53.779625,10508003,0,4.041452,1.041979,1.000000,1.107913,48.541391,53.779625,0,1.041979,4.041452,53.779625,5737379,30,0.0,2694.000000,15986.000000
4,53.076979,10714831,0,3.464102,1.069917,1.000000,1.107615,47.920072,53.076979,0,1.069917,3.464102,53.076979,4403184,16,53638.0,64.000000,23682.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546579,0.000000,10711987,1,0.534522,2.075304,1.000000,0.693147,0.000000,0.000000,1,2.075304,0.534522,0.000000,3303594,4,1422.0,416.903846,2765.673077
546580,0.000000,11143735,0,0.000000,1.791759,1.000000,0.693147,0.000000,0.000000,0,1.791759,0.000000,0.000000,2152513,12751,17568.0,7163.777778,28322.666667
546581,0.000000,11143737,0,0.000000,1.386294,1.000000,0.693147,0.000000,0.000000,0,1.386294,0.000000,0.000000,5912473,83,5306.0,254.000000,2976.000000
546582,0.000000,11143738,0,0.000000,0.693147,1.000000,1.791759,0.000000,0.000000,0,0.693147,0.000000,0.000000,4782758,215,20315.0,442.250000,14729.000000


# BaseLine 

In [ ]:
nFraud = df[df['FraudId']==0].shape[0]
print(df.shape[0] , df.shape[0] - nFraud)
BaseLine_acc = nFraud/df.shape[0]*100
print('The BaseLine accuracy is: %', BaseLine_acc)

546584 74114
The BaseLine accuracy is: % 86.4405105162244


# The Data and Model

In [ ]:
MyData1

,STD_DI,OrderId,FraudId,STD,DI,DIsum,STD_avg,STD_DI_avg,CustomerId
0,60.192635,10261118,0,0.957427,2.661840,2.661840,0.957427,60.192635,5608657
1,56.388985,10861839,0,1.366260,1.648847,1.648847,1.366260,56.388985,2839661
2,55.056967,11216905,0,0.421637,2.274241,2.274241,0.421637,55.056967,4892643
3,53.779625,10508003,0,4.041452,1.041979,1.041979,4.041452,53.779625,5737379
4,53.076979,10714831,0,3.464102,1.069917,1.069917,3.464102,53.076979,4403184
...,...,...,...,...,...,...,...,...,...
546579,0.000000,10711987,1,0.534522,2.075304,2.075304,0.534522,0.000000,3303594
546580,0.000000,11143735,0,0.000000,1.791759,1.791759,0.000000,0.000000,2152513
546581,0.000000,11143737,0,0.000000,1.386294,1.386294,0.000000,0.000000,5912473
546582,0.000000,11143738,0,0.000000,0.693147,0.693147,0.000000,0.000000,4782758


In [ ]:
########### The test-train data #################

dfT1 = MyData1.drop(columns = ['CustomerId','OrderId','STD_DI','STD_DI_avg']).dropna()

shuffled1=dfT1.sample(frac=1).reset_index(drop=True)
DF1= shuffled1.iloc[:,:]

y1 = DF1['FraudId']
DF1_T = DF1.drop(columns={'FraudId'})

X1_train, X1_test, y1_train, y1_test = train_test_split(DF1_T, y1, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf1 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5)
rf1.fit(X1_train, y1_train)

y1_prediction = rf1.predict(X1_test)

rf1.score(X1_train, y1_train)
acc_rf1 = round(rf1.score(X1_train, y1_train) * 100, 2)

acc_rf1


y1_train_pred =cross_val_predict(rf1, X1_train, y1_train, cv=10)
print("M1 CV Precision: %f "%precision_score(y1_train, y1_train_pred))
print("M1 CV Recall: %f "%recall_score(y1_train, y1_train_pred))
print("M1 CV F1: %f"% f1_score(y1_train, y1_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y1_test_pred=rf1.predict(X1_test)
# _scaled)

print("M1 Precision: %f "%precision_score(y1_test, y1_test_pred))
print("M1 Recall: %f "%recall_score(y1_test, y1_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y1_test, y1_test_pred))


auc1 = roc_auc_score(y1_test, y1_test_pred)

T = y1.shape[0]
A = rf1.score(X1_train, y1_train)
P = precision_score(y1_test, y1_test_pred)
R = recall_score(y1_test, y1_test_pred)

TP1 = (1-A)*T/(1/P + 1/R + 2)
FP1 = (TP1/P) - TP1
FN1 = (TP1/R) - TP1
TN1 = T - FN1 -FP1 - TP1

M1_FI = rf1.feature_importances_
M1_f_beta = fbeta_score(y1_test, y1_test_pred, average='macro', beta=0.5)

print('M2 Accuracy:', acc_rf1, 
      ',\n M2 AUC: ',auc1 ,
      ',\n M2 True Positive:' , TP1 , 
      ',\n M2 False Positive:' , FP1 , 
      ',\n M2 True Negative:', TN1 ,
      ',\n M2 False Negative:', FN1,
     ',\n M2 Features: \n', X1_train.columns, 
     ',\n M2 Feature Importance: \n' , M1_FI,
     ',\n M2 f_0.5:', M1_f_beta)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   20.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   20.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M1 CV Precision: 0.000000 
M1 CV Recall: 0.000000 
M1 CV F1: 0.000000


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   10.0s finished
C:\Users\Mahdieh.Fatemeh\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


M1 Precision: 0.000000 
M1 Recall: 0.000000 
Let's see the confuision matrix:
 [[155961      0]
 [ 24412      0]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   20.5s finished
C:\Users\Mahdieh.Fatemeh\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-58-54a7158a5283>:48: RuntimeWarning: divide by zero encountered in double_scalars
  TP1 = (1-A)*T/(1/P + 1/R + 2)
<ipython-input-58-54a7158a5283>:49: RuntimeWarning: invalid value encountered in double_scalars
  FP1 = (TP1/P) - TP1
<ipython-input-58-54a7158a5283>:50: RuntimeWarning: invalid value encountered in double_scalars
  FN1 = (TP1/R) - TP1


M2 Accuracy: 86.43 ,
 M2 AUC:  0.5 ,
 M2 True Positive: 0.0 ,
 M2 False Positive: nan ,
 M2 True Negative: nan ,
 M2 False Negative: nan ,
 M2 Features: 
 Index(['STD', 'DI', 'DIsum', 'STD_avg'], dtype='object') ,
 M2 Feature Importance: 
 [0.04177642 0.45790583 0.46128365 0.0390341 ] ,
 M2 f_0.5: 0.444357133658441


In [ ]:
########### The test-train data #################

dfT2 = MyData2.drop(columns = ['CustomerId','OrderId']).dropna()

shuffled2=dfT2.sample(frac=1).reset_index(drop=True)
DF2= shuffled2.iloc[:,:]

y2 = DF2['FraudId']
DF2_T = DF2.drop(columns={'FraudId'})

X2_train, X2_test, y2_train, y2_test = train_test_split(DF2_T, y2, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf2 = RandomForestClassifier(n_estimators=400, verbose=1 , max_depth = 6)
rf2.fit(X2_train, y2_train)

y2_prediction = rf2.predict(X2_test)

rf2.score(X2_train, y2_train)
acc_rf2 = round(rf2.score(X2_train, y2_train) * 100, 2)


####################################################
####################################################


y2_train_pred =cross_val_predict(rf2, X2_train, y2_train, cv=10)
print("M2 CV Precision: %f "%precision_score(y2_train, y2_train_pred))
print("M2 CVRecall: %f "%recall_score(y2_train, y2_train_pred))
print("M2 CV F1: %f"% f1_score(y2_train, y2_train_pred))

############################################# 
# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y2_test_pred=rf2.predict(X2_test)

print("M2 Precision: %f "%precision_score(y2_test, y2_test_pred))
print("M2 Recall: %f "%recall_score(y2_test, y2_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y2_test, y2_test_pred))


auc2 = roc_auc_score(y2_test, y2_test_pred)

T = y2.shape[0]
A = rf2.score(X2_train, y2_train)
P = precision_score(y2_test, y2_test_pred)
R = recall_score(y2_test, y2_test_pred)

TP2 = (1-A)*T/(1/P + 1/R + 2)
FP2 = (TP2/P) - TP2
FN2 = (TP2/R) - TP2
TN2 = T - FN2 -FP2 - TP2

M2_FI = rf2.feature_importances_
M2_f_beta = fbeta_score(y2_test, y2_test_pred, average='macro', beta=0.5)

print('M2 Accuracy:', acc_rf2, 
      ',\n M2 AUC: ',auc2 ,
      ',\n M2 True Positive:' , TP2 , 
      ',\n M2 False Positive:' , FP2 , 
      ',\n M2 True Negative:', TN2 ,
      ',\n M2 False Negative:', FN2,
     ',\n M2 Features: \n', X2_train.columns, 
     ',\n M2 Feature Importance: \n' , M2_FI,
     ',\n M2 f_0.5:', M2_f_beta)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M2 CV Precision: 0.000000 
M2 CVRecall: 0.000000 
M2 CV F1: 0.000000


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    5.5s finished
C:\Users\Mahdieh.Fatemeh\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


M2 Precision: 0.000000 
M2 Recall: 0.000000 
Let's see the confuision matrix:
 [[155825      0]
 [ 24548      0]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.1s finished
C:\Users\Mahdieh.Fatemeh\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-13-f9475609c2cb>:50: RuntimeWarning: divide by zero encountered in double_scalars
  TP2 = (1-A)*T/(1/P + 1/R + 2)
<ipython-input-13-f9475609c2cb>:51: RuntimeWarning: invalid value encountered in double_scalars
  FP2 = (TP2/P) - TP2
<ipython-input-13-f9475609c2cb>:52: RuntimeWarning: invalid value encountered in double_scalars
  FN2 = (TP2/R) - TP2


M2 Accuracy: 86.47 ,
 M2 AUC:  0.5 ,
 M2 True Positive: 0.0 ,
 M2 False Positive: nan ,
 M2 True Negative: nan ,
 M2 False Negative: nan ,
 M2 Features: 
 Index(['STD_DI', 'STD', 'DI', 'DIsum', 'STD_avg', 'STD_DI_avg', 'ToMinPrice',
       'DisProfit'],
      dtype='object') ,
 M2 Feature Importance: 
 [0.00169015 0.02627805 0.2511935  0.24188462 0.02666959 0.00179234
 0.18203423 0.26845752] ,
 M2 f_0.5: 0.44403847184085116


In [ ]:
########### The test-train data #################

dfT2 = MyData2.drop(columns = ['CustomerId','OrderId']).dropna()

shuffled2=dfT2.sample(frac=1).reset_index(drop=True)
DF2= shuffled2.iloc[:,:]

y2 = DF2['FraudId']
DF2_T = DF2.drop(columns={'FraudId'})

X2_train, X2_test, y2_train, y2_test = train_test_split(DF2_T, y2, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf2 = RandomForestClassifier(n_estimators=400, verbose=1 , max_depth = 6)
rf2.fit(X2_train, y2_train)

y2_prediction = rf2.predict(X2_test)

rf2.score(X2_train, y2_train)
acc_rf2 = round(rf2.score(X2_train, y2_train) * 100, 2)


####################################################
####################################################


y2_train_pred =cross_val_predict(rf2, X2_train, y2_train, cv=10)
print("M2 CV Precision: %f "%precision_score(y2_train, y2_train_pred))
print("M2 CVRecall: %f "%recall_score(y2_train, y2_train_pred))
print("M2 CV F1: %f"% f1_score(y2_train, y2_train_pred))

############################################# 
# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y2_test_pred=rf2.predict(X2_test)

print("M2 Precision: %f "%precision_score(y2_test, y2_test_pred))
print("M2 Recall: %f "%recall_score(y2_test, y2_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y2_test, y2_test_pred))


auc2 = roc_auc_score(y2_test, y2_test_pred)

T = y2.shape[0]
A = rf2.score(X2_train, y2_train)
P = precision_score(y2_test, y2_test_pred)
R = recall_score(y2_test, y2_test_pred)

TP2 = (1-A)*T/(1/P + 1/R + 2)
FP2 = (TP2/P) - TP2
FN2 = (TP2/R) - TP2
TN2 = T - FN2 -FP2 - TP2

M2_FI = rf2.feature_importances_
M2_f_beta = fbeta_score(y2_test, y2_test_pred, average='macro', beta=0.5)

print('M2 Accuracy:', acc_rf2, 
      ',\n M2 AUC: ',auc2 ,
      ',\n M2 True Positive:' , TP2 , 
      ',\n M2 False Positive:' , FP2 , 
      ',\n M2 True Negative:', TN2 ,
      ',\n M2 False Negative:', FN2,
     ',\n M2 Features: \n', X2_train.columns, 
     ',\n M2 Feature Importance: \n' , M2_FI,
     ',\n M2 f_0.5:', M2_f_beta)

In [ ]:
# counter = Counter(y1_train)
# label,_ = counter.items()
# counter

In [ ]:
# counter = 
# # scatter plot of examples by class label
# for label in counter.items():
#     row_ix = where(y == label)[0]
#     pyplot.scatter(X1_train[row_ix, 0], X1_train[row_ix, 1])
# pyplot.legend()
# pyplot.show()

In [ ]:
label

(0, 316509)

In [ ]:
########### The test-train data #################

dfT3 = MyData3.drop(columns = ['CustomerId','OrderId','STD_DI',
                                'STD_DI_avg', 'NumDis_DI', 'NumDis_STD','NumDis_STD_DI'])

shuffled3=dfT3.sample(frac=1).reset_index(drop=True)
DF3= shuffled3.iloc[:,:]

y3 = DF3['FraudId']
DF3_T = DF3.drop(columns={'FraudId'})

X3_train, X3_test, y3_train, y3_test = train_test_split(DF3_T, y3, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf3 = RandomForestClassifier(n_estimators=400, verbose=1 , max_depth = 6)
rf3.fit(X3_train, y3_train)

y3_prediction = rf3.predict(X3_test)

rf3.score(X3_train, y3_train)
acc_rf3 = round(rf3.score(X3_train, y3_train) * 100, 2)
print('M3 accuracy is:' , acc_rf3)

y3_train_pred =cross_val_predict(rf3, X3_train, y3_train, cv=10)
print("M3 CV Precision: %f "%precision_score(y3_train, y3_train_pred))
print("M3 CVRecall: %f "%recall_score(y3_train, y3_train_pred))
print("M3 CV F1: %f"% f1_score(y3_train, y3_train_pred))

############################################ X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y3_test_pred=rf3.predict(X3_test)

print("M3 Precision: %f "%precision_score(y3_test, y3_test_pred))
print("M3 Recall: %f "%recall_score(y3_test, y3_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y3_test, y3_test_pred))


auc3 = roc_auc_score(y3_test, y3_test_pred)

T = y3.shape[0]
A = rf3.score(X3_train, y3_train)
P = precision_score(y3_test, y3_test_pred)
R = recall_score(y3_test, y3_test_pred)

TP3 = (1-A)*T/(1/P + 1/R + 2)
FP3 = (TP3/P) - TP3
FN3 = (TP3/R) - TP3
TN3 = T - FN3 -FP3 - TP3

M3_FI = rf3.feature_importances_
M3_f_beta = fbeta_score(y3_test, y3_test_pred, average='macro', beta=0.5)

print('M2 Accuracy:', acc_rf3, 
      ',\n M2 AUC: ',auc3 ,
      ',\n M2 True Positive:' , TP3 , 
      ',\n M2 False Positive:' , FP3 , 
      ',\n M2 True Negative:', TN3 ,
      ',\n M2 False Negative:', FN3,
     ',\n M2 Features: \n', X3_train.columns, 
     ',\n M2 Feature Importance: \n' , M3_FI,
     ',\n M2 f_0.5:', M3_f_beta)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.1s finished


M3 accuracy is: 86.46


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M3 CV Precision: 0.790698 
M3 CVRecall: 0.002737 
M3 CV F1: 0.005456


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    5.6s finished


M3 Precision: 0.671429 
M3 Recall: 0.001924 
Let's see the confuision matrix:
 [[155918     23]
 [ 24385     47]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   11.3s finished


M2 Accuracy: 86.46 ,
 M2 AUC:  0.5008881074673655 ,
 M2 True Positive: 141.393806391385 ,
 M2 False Positive: 69.19271376599693 ,
 M2 True Negative: 473014.0949935889 ,
 M2 False Negative: 73359.31848625369 ,
 M2 Features: 
 Index(['STD_DI', 'STD', 'DI', 'DIsum', 'STD_avg', 'STD_DI_avg', 'ToMinPrice',
       'DisProfit', 'NumDis_NumPur', 'NumDis_DI', 'NumDis_STD',
       'NumDis_STD_DI'],
      dtype='object') ,
 M2 Feature Importance: 
 [0.00065102 0.01274079 0.19359125 0.19488241 0.01464275 0.00066388
 0.13201578 0.18623278 0.2619586  0.00112713 0.0007547  0.0007389 ] ,
 M2 f_0.5: 0.4491413301472765


In [ ]:
MyData4['DI_TimeToShoppingCard'] = MyData4['DI']*MyData4['TimeToShoppingCart']

In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI',
                                'STD_DI_avg', 'NumDis_DI', 'NumDis_STD','NumDis_STD_DI']).dropna()

shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=600, verbose=1 , max_depth = 6)
rf4.fit(X4_train, y4_train)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train, y4_train)
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4 ,
      '\n M2 True Positive:' , TP4 , 
      ',\n M2 False Positive:' , FP4 , 
      ',\n M2 True Negative:', TN4 ,
      ',\n M2 False Negative:', FN4,
     '\n M2 Features: \n', X4_train.columns, 
     '\n M2 Feature Importance: \n' , M4_FI)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  3.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    8.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   17.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   17.9s finished


M4 accuracy: 86.72


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M4 CV Precision: 0.662617 
M4 CVRecall: 0.042781 
M4 CV F1: 0.080373


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    8.8s finished


M4 Precision: 0.668136 
M4 Recall: 0.043491 
Let's see the confuision matrix:
 [[155450    527]
 [ 23335   1061]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   17.5s finished


f_0.3: 0.5921958294010407
M2 Accuracy: 86.72 
 M2 AUC:  0.520056016457953 
 M2 True Positive: 2739.859415385531 ,
 M2 False Positive: 1360.8915286599195 ,
 M2 True Negative: 482224.4182755386 ,
 M2 False Negative: 60258.83078041599 
 M2 Features: 
 Index(['STD', 'DI', 'DIsum', 'STD_avg', 'ToMinPrice', 'DisProfit',
       'NumDis_NumPur', 'TimeToShoppingCart', 'TimeToPreviousOrder',
       'DI_TimeToShoppingCard'],
      dtype='object') 
 M2 Feature Importance: 
 [0.00426559 0.06992168 0.08439206 0.0041635  0.08089335 0.11220345
 0.16511887 0.12478773 0.2672197  0.08703406]


In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI_avg', 'NumDis_DI', 'NumDis_STD','NumDis_STD_DI','STD','STD_avg'])
shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=600, verbose=1 , max_depth = 5)
rf4.fit(X4_train, y4_train)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train, y4_train)
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4 ,
      '\n M2 True Positive:' , TP4 , 
      ',\n M2 False Positive:' , FP4 , 
      ',\n M2 True Negative:', TN4 ,
      ',\n M2 False Negative:', FN4,
     '\n M2 Features: \n', X4_train.columns, 
     '\n M2 Feature Importance: \n' , M4_FI)



In [ ]:
rfc_balanced = RandomForestClassifier(random_state = 42, n_jobs = -1, class_weight = "balanced")

# Handling imbalanced data

In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI_avg', 'NumDis_DI', 'NumDis_STD','NumDis_STD_DI','STD','STD_avg'])

shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5, class_weight = "balanced")
rf4.fit(X4_train, y4_train)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train, y4_train)
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4 ,
      '\n M2 True Positive:' , TP4 , 
      ',\n M2 False Positive:' , FP4 , 
      ',\n M2 True Negative:', TN4 ,
      ',\n M2 False Negative:', FN4,
     '\n M2 Features: \n', X4_train.columns, 
     '\n M2 Feature Importance: \n' , M4_FI)



In [ ]:
def rus(X_train: pd.DataFrame, y_train: pd.DataFrame, sampling_strategy: float):
    
    """ Simple implementation of RandomUnderSampling """
    
    train=pd.concat([X_train,y_train],axis=1)
    
    train_maj=train[train.FraudId==0]
    train_min=train[train.FraudId==1]
    
    train_maj_rus=train_maj.sample(int(1/sampling_strategy*len(train_min)),random_state=randint(1,100000))
    
    train_rus=pd.concat([train_maj_rus,train_min])
    
    X_train_rus= train_rus.drop('FraudId',axis=1)
    y_train_rus= train_rus.FraudId
    
    return X_train_rus,y_train_rus

In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI', 'STD', 'NumDis_STD','STD_avg', 'STD_DI_avg',
       'NumDis_STD_DI'])

shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

X4_train_bl,y4_train_bl= rus(X4_train,y4_train,sampling_strategy=0.2)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5)
rf4.fit(X4_train_bl, y4_train_bl)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train_bl, y4_train_bl)
acc_rf4 = round(rf4.score(X4_train_bl, y4_train_bl) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train_bl, y4_train_bl, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train_bl, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train_bl, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train_bl, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train_bl, y4_train_bl)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4,
      '\n M2 True Positive:' ,TP4, 
      ',\n M2 False Positive:' ,FP4, 
      ',\n M2 True Negative:',TN4,
      ',\n M2 False Negative:',FN4,
     '\n M2 Features: \n',X4_train_bl.columns, 
     '\n M2 Feature Importance: \n' ,M4_FI)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   15.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   16.1s finished


M4 accuracy: 83.88


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M4 CV Precision: 0.681146 
M4 CVRecall: 0.060518 
M4 CV F1: 0.111161


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   10.3s finished


M4 Precision: 0.641986 
M4 Recall: 0.061970 
Let's see the confuision matrix:
 [[154897    851]
 [ 23099   1526]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   16.7s finished


f_0.3: 0.6207146048649637
M2 Accuracy: 83.88 
 M2 AUC:  0.5282527942769453 
 M2 True Positive: 4473.050308931578 ,
 M2 False Positive: 2494.473009764595 ,
 M2 True Negative: 471908.0958909955 ,
 M2 False Negative: 67708.38079030834 
 M2 Features: 
 Index(['DI', 'DIsum', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur', 'NumDis_DI',
       'TimeToShoppingCart', 'TimeToPreviousOrder', 'DI_TimeToShoppingCard'],
      dtype='object') 
 M2 Feature Importance: 
 [8.12896263e-02 8.17609180e-02 7.42991764e-02 1.03077972e-01
 1.46837835e-01 1.00710851e-04 1.32495973e-01 2.74442178e-01
 1.05695610e-01]


In [ ]:
# !pip3 install imblearn --user

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.8.0-py3-none-any.whl (206 kB)
  Using cached scikit_learn-0.24.1-cp38-cp38-win_amd64.whl (6.9 MB)


In [ ]:
!pip3 install imbalanced-learn --user

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=42)


In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI', 'STD', 'NumDis_STD','STD_avg', 'STD_DI_avg',
       'NumDis_STD_DI'])

shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

#######################################
DF4_T_bl, y4_bl = ros.fit_resample(DF4_T, y4)
print('Original dataset shape', Counter(y4))
print('Resample dataset shape', Counter(y4_bl))
#######################################

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T_bl, y4_bl, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5)
rf4.fit(X4_train, y4_train)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train, y4_train)
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4 ,
      '\n M2 True Positive:' , TP4 , 
      ',\n M2 False Positive:' , FP4 , 
      ',\n M2 True Negative:', TN4 ,
      ',\n M2 False Negative:', FN4,
     '\n M2 Features: \n', X4_train.columns, 
     '\n M2 Feature Importance: \n' , M4_FI)



Original dataset shape Counter({0: 472470, 1: 74114})
Resample dataset shape Counter({0: 472470, 1: 472470})


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
X = MyData4.drop(columns=['FraudId'])
y = MyData4['FraudId']

In [ ]:
a = counter.items()


dict_items([(0, 472470), (1, 74114)])

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
oversample = SMOTE()
# Xb, yb = oversample.fit_resample(X, y)

In [ ]:
MyData4.columns

Index(['STD_DI', 'OrderId', 'FraudId', 'STD', 'DI', 'DIsum', 'STD_avg',
       'STD_DI_avg', 'CustomerId', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur',
       'NumDis_DI', 'NumDis_STD', 'NumDis_STD_DI', 'TimeToShoppingCart',
       'TimeToPreviousOrder', 'DI_TimeToShoppingCard'],
      dtype='object')

In [ ]:
Avgs4 = MyData4.groupby('CustomerId').agg({'DI':'mean', 'ToMinPrice':'mean', 
                                           'DisProfit':'mean', 'TimeToShoppingCart':'mean',
                                           'TimeToPreviousOrder':'mean'
}).drop_duplicates().reset_index().rename(columns = {'DI':'DI_avg', 'ToMinPrice':'ToMinPrice_avg', 
                                           'DisProfit':'DisProfit_avg', 'TimeToShoppingCart':'TimeToShoppingCart_avg',
                                           'TimeToPreviousOrder':'TimeToPreviousOrder_avg'
})

In [ ]:
MyData4_avg = MyData4.merge(Avgs4 , left_on = 'CustomerId' , right_on='CustomerId').drop_duplicates().reset_index(drop=True)

In [ ]:
MyData4_avg.columns

Index(['STD_DI', 'OrderId', 'FraudId', 'STD', 'DI', 'DIsum', 'STD_avg',
       'STD_DI_avg', 'CustomerId', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur',
       'NumDis_DI', 'NumDis_STD', 'NumDis_STD_DI', 'TimeToShoppingCart',
       'TimeToPreviousOrder', 'DI_TimeToShoppingCard', 'DI_avg',
       'ToMinPrice_avg', 'DisProfit_avg', 'TimeToShoppingCart_avg',
       'TimeToPreviousOrder_avg'],
      dtype='object')

In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI', 'STD', 'NumDis_STD','STD_avg', 'STD_DI_avg',
       'NumDis_STD_DI'])

shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

X4_train_bl,y4_train_bl= oversample.fit_resample(X4_train,y4_train)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5)
rf4.fit(X4_train_bl, y4_train_bl)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train_bl, y4_train_bl)
acc_rf4 = round(rf4.score(X4_train_bl, y4_train_bl) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train_bl, y4_train_bl, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train_bl, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train_bl, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train_bl, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train_bl, y4_train_bl)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4,
      '\n M2 True Positive:' ,TP4, 
      ',\n M2 False Positive:' ,FP4, 
      ',\n M2 True Negative:',TN4,
      ',\n M2 False Negative:',FN4,
     '\n M2 Features: \n',X4_train_bl.columns, 
     '\n M2 Feature Importance: \n' ,M4_FI)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  9.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   33.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   33.0s finished


M4 accuracy: 76.81


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  7.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  7.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  7.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M4 CV Precision: 0.781336 
M4 CVRecall: 0.736474 
M4 CV F1: 0.758242


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.7s finished


M4 Precision: 0.302014 
M4 Recall: 0.581167 
Let's see the confuision matrix:
 [[123416  32649]
 [ 10181  14127]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   33.4s finished


f_0.3: 0.6128137135445942
M2 Accuracy: 76.81 
 M2 AUC:  0.6859826998799952 
 M2 True Positive: 18029.18607861777 ,
 M2 False Positive: 41667.36718912661 ,
 M2 True Negative: 473894.23207476235 ,
 M2 False Negative: 12993.214657493274 
 M2 Features: 
 Index(['DI', 'DIsum', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur', 'NumDis_DI',
       'TimeToShoppingCart', 'TimeToPreviousOrder', 'DI_TimeToShoppingCard'],
      dtype='object') 
 M2 Feature Importance: 
 [0.14023901 0.13576953 0.02677074 0.04533347 0.3969552  0.01787995
 0.05213755 0.1635685  0.02134605]


In [ ]:
# ########### The test-train data #################

# dfT4 = MyData4_avg.drop(columns = ['CustomerId','OrderId','STD_DI', 'STD', 'NumDis_STD','STD_avg', 'STD_DI_avg',
#        'NumDis_STD_DI'])

# shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
# DF4= shuffled4.iloc[:,:]

# y4 = DF4['FraudId']
# DF4_T = DF4.drop(columns={'FraudId'})

# X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

# # X4_train_bl,y4_train_bl= oversample.fit_resample(X4_train,y4_train)

# #################### THe Model #####################

# rf4 = RandomForestClassifier(n_estimators=800, verbose=1 , max_depth = 5)
# rf4.fit(X4_train, y4_train)

# y4_prediction = rf4.predict(X4_test)

print('test accuracy:' ,rf4.score(X4_train, y4_train))
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4,
      '\n M2 True Positive:' ,TP4, 
      ',\n M2 False Positive:' ,FP4, 
      ',\n M2 True Negative:',TN4,
      ',\n M2 False Negative:',FN4,
     '\n M2 Features: \n',X4_train_bl.columns, 
     '\n M2 Feature Importance: \n' ,M4_FI)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   19.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


test accuracy: 0.8643312660733575


[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   19.7s finished


M4 accuracy: 86.43


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  3.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  3.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  3.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M4 CV Precision: 0.826087 
M4 CVRecall: 0.003448 
M4 CV F1: 0.006868


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.5s finished


M4 Precision: 0.837838 
M4 Recall: 0.002528 
Let's see the confuision matrix:
 [[155112     12]
 [ 24460     62]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:   19.6s finished


f_0.3: 0.4516177129192324
M2 Accuracy: 86.43 
 M2 AUC:  0.5012254922141042 
 M2 True Positive: 185.2358985942196 ,
 M2 False Positive: 35.85210940533284 ,
 M2 True Negative: 471080.36232079705 ,
 M2 False Negative: 73078.54967120341 
 M2 Features: 
 Index(['DI', 'DIsum', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur', 'NumDis_DI',
       'TimeToShoppingCart', 'TimeToPreviousOrder', 'DI_TimeToShoppingCard',
       'DI_avg', 'ToMinPrice_avg', 'DisProfit_avg', 'TimeToShoppingCart_avg',
       'TimeToPreviousOrder_avg'],
      dtype='object') 
 M2 Feature Importance: 
 [4.54825801e-02 4.47011012e-02 3.94281814e-02 5.94642106e-02
 8.51545268e-02 4.39891379e-05 8.49523333e-02 1.44498101e-01
 7.53202496e-02 1.63922572e-01 2.61794458e-02 4.55177632e-02
 1.15167127e-02 1.73818233e-01]


In [ ]:
X4_train.columns

Index(['DI', 'DIsum', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur', 'NumDis_DI',
       'TimeToShoppingCart', 'TimeToPreviousOrder', 'DI_TimeToShoppingCard',
       'DI_avg', 'ToMinPrice_avg', 'DisProfit_avg', 'TimeToShoppingCart_avg',
       'TimeToPreviousOrder_avg'],
      dtype='object')

In [ ]:
########### The test-train data #################

dfT4 = MyData4.drop(columns = ['CustomerId','OrderId','STD_DI_avg', 'NumDis_DI', 'NumDis_STD','NumDis_STD_DI','STD','STD_avg'])
shuffled4=dfT4.sample(frac=1).reset_index(drop=True)
DF4= shuffled4.iloc[:,:]

y4 = DF4['FraudId']
DF4_T = DF4.drop(columns={'FraudId'})

X4_train, X4_test, y4_train, y4_test = train_test_split(DF4_T, y4, test_size=0.33,random_state = 0)

#################### THe Model #####################

rf4 = RandomForestClassifier(n_estimators=600, verbose=1 , max_depth = 5)
rf4.fit(X4_train, y4_train)

y4_prediction = rf4.predict(X4_test)

rf4.score(X4_train, y4_train)
acc_rf4 = round(rf4.score(X4_train, y4_train) * 100, 2)

print('M4 accuracy:' , acc_rf4)

y4_train_pred =cross_val_predict(rf4, X4_train, y4_train, cv=10)
print("M4 CV Precision: %f "%precision_score(y4_train, y4_train_pred))
print("M4 CVRecall: %f "%recall_score(y4_train, y4_train_pred))
print("M4 CV F1: %f"% f1_score(y4_train, y4_train_pred))

############################################# X_test_scaled=preprocessing_pipe.fit_transform(X_test)
y4_test_pred=rf4.predict(X4_test)


print("M4 Precision: %f "%precision_score(y4_test, y4_test_pred))
print("M4 Recall: %f "%recall_score(y4_test, y4_test_pred))
# print("F1: %f"% f1_score(y_train, y_prediction))
print("Let's see the confuision matrix:\n",confusion_matrix(y4_test, y4_test_pred))


auc4 = roc_auc_score(y4_test, y4_test_pred)

T = y4.shape[0]
A = rf4.score(X4_train, y4_train)
P = precision_score(y4_test, y4_test_pred)
R = recall_score(y4_test, y4_test_pred)

TP4 = (1-A)*T/(1/P + 1/R + 2)
FP4 = (TP4/P) - TP4
FN4 = (TP4/R) - TP4
TN4 = T - FN4 -FP4 - TP4

M4_FI = rf4.feature_importances_
M4_f1_beta = fbeta_score(y4_test, y4_test_pred, average='macro', beta=0.3)
print('f_0.3:', M4_f1_beta)

print('M2 Accuracy:', acc_rf4, 
      '\n M2 AUC: ',auc4 ,
      '\n M2 True Positive:' , TP4 , 
      ',\n M2 False Positive:' , FP4 , 
      ',\n M2 True Negative:', TN4 ,
      ',\n M2 False Negative:', FN4,
     '\n M2 Features: \n', X4_train.columns, 
     '\n M2 Feature Importance: \n' , M4_FI)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   15.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   15.0s finished


M4 accuracy: 86.64


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

M4 CV Precision: 0.679253 
M4 CVRecall: 0.032868 
M4 CV F1: 0.062702


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    7.4s finished


M4 Precision: 0.667278 
M4 Recall: 0.029948 
Let's see the confuision matrix:
 [[155701    363]
 [ 23581    728]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   14.9s finished


f_0.3: 0.5599357199253003
M2 Accuracy: 86.64 
 M2 AUC:  0.513810893570927 
 M2 True Positive: 1979.8226938138878 ,
 M2 False Positive: 987.1918102396171 ,
 M2 True Negative: 479487.5913437139 ,
 M2 False Negative: 64129.39415223254 
 M2 Features: 
 Index(['STD_DI', 'DI', 'DIsum', 'ToMinPrice', 'DisProfit', 'NumDis_NumPur',
       'TimeToShoppingCart', 'TimeToPreviousOrder', 'DI_TimeToShoppingCard'],
      dtype='object') 
 M2 Feature Importance: 
 [5.92708089e-05 7.89183793e-02 7.73002709e-02 7.62241738e-02
 9.92362345e-02 1.55242107e-01 1.41483162e-01 2.70376145e-01
 1.01160256e-01]


# Saving Model

In [ ]:
import pickle

In [ ]:
Pkl_Filename = "RF_Fraud_V1_customer_avg.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(rf4, file)

In [ ]:
with open(Pkl_Filename, 'rb') as file:  
    RF_Fraud_customer = pickle.load(file)

RF_Fraud_customer

RandomForestClassifier(max_depth=5, n_estimators=800, verbose=1)

In [ ]:
score = RF_Fraud_customer.score(X4_test, y4_test) 
score

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:    9.7s finished


0.8637765383031072

# Input data function

In [ ]:
FraudAnalysisData_complete = pd.read_csv('FraudAnalysisData_complete.csv')


In [ ]:
def DI(basket):
    len_basket = len(basket)
    pi_basket = [x / len_basket for x in basket]

    di_basket = sum([-x*np.log(x) for x in pi_basket])
    return di_basket

def DiscountFraction(CountDiscount , CountOrder):
    return CountDiscount/CountOrder

def DiscountProfitNormalized(OrderAmount , CACDiscountAmount):
    return (OrderAmount - CACDiscountAmount)/OrderAmount

def ToMinPrice(OrderAmount , CACMinOrderAmount):
    return (OrderAmount - CACMinOrderAmount)/OrderAmount

# grouped by customer

In [ ]:
df_c = df.groupby('CustomerId').agg({
'FraudId_avg':'first'    
}).reset_index(drop=False)
nFraud_c = df_c[df_c['FraudId_avg']==0].shape[0]
print(df_c.shape[0] , nFraud_c)
BaseLine_c_acc = nFraud_c/df_c.shape[0]*100
print('The BaseLine accuracy is: %', BaseLine_c_acc)

386752 353998
The BaseLine accuracy is: % 91.5310069501903


# Adding Location 

In [ ]:
Basket_info

,OrderId,OrderState,Latitude,Longitude,CustomerId,TimeToShoppingCart,TimeToPreviousOrder
0,1250951,5,36.321957,59.523514,87,21,NaN
1,1292950,5,36.321957,59.523514,87,4913,7679.0
2,1379188,5,36.325459,59.529143,87,20,6126.0
3,1426710,5,36.321957,59.523514,87,13,9702.0
4,1473326,5,36.321957,59.523514,87,1075,12574.0
...,...,...,...,...,...,...,...
8146205,12306964,4,32.461146,48.349562,6649529,2,NaN
8146206,12307227,5,35.669596,51.302731,6649614,1,NaN
8146207,12307889,4,35.812608,50.900292,6649714,9,NaN
8146208,12307822,4,30.257237,57.052058,6649796,3,NaN


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Okala")
location = geolocator.reverse("32.461146, 48.349562")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

موسوی (گیتی), کوی شهدا, اندیمشک, دهستان حومه, بخش مرکزی, شهرستان اندیمشک, استان خوزستان, ایران
(32.46115305871785, 48.34948144619332)
{'place_id': 218030451, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 618328675, 'lat': '32.46115305871785', 'lon': '48.34948144619332', 'display_name': 'موسوی (گیتی), کوی شهدا, اندیمشک, دهستان حومه, بخش مرکزی, شهرستان اندیمشک, استان خوزستان, ایران', 'address': {'road': 'موسوی (گیتی)', 'neighbourhood': 'کوی شهدا', 'city': 'اندیمشک', 'district': 'بخش مرکزی', 'county': 'شهرستان اندیمشک', 'state': 'استان خوزستان', 'country': 'ایران', 'country_code': 'ir'}, 'boundingbox': ['32.461027', '32.4617972', '48.3493846', '48.3495189']}


In [ ]:
from geopy.distance import geodesic
newport_ri = (32.461146, 48.349562)
cleveland_oh = (35.669596, 51.302731)
print(geodesic(newport_ri, cleveland_oh).kilometers)

448.25169320006415


In [ ]:
from geopy.distance import great_circle
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(great_circle(newport_ri, cleveland_oh).kilometers)

864.2144943393627
